In [ ]:
#| default_exp ets

In [ ]:
#| export
import math
import os
import warnings
from collections import namedtuple
from typing import Tuple

import numpy as np
from numba import njit
from statsmodels.tsa.seasonal import seasonal_decompose

# ETS Model

## etscalc

In [ ]:
#| export
# Global variables 
NONE = 0
SIMPLE = 1
PARTIAL = 2
FULL = 3
TOL = 1.0e-10
HUGEN = 1.0e10
NA = -99999.0
smalno = np.finfo(float).eps
NOGIL = os.environ.get('NUMBA_RELEASE_GIL', 'False').lower() in ['true']
CACHE = os.environ.get('NUMBA_CACHE', 'False').lower() in ['true']

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def cescalc(y, n, 
            states, # states
            m, 
            season, 
            alpha_0, alpha_1,
            beta_0, beta_1, 
            e, amse, nmse):
    f = np.zeros(nmse)
    denom = np.zeros(nmse)
    m = 1 if season == NONE else m
    lik = 0.
    lik2 = 0.
    amse[:nmse] = 0.
    for i in range(m, n + m):
        # one step forecast 
        forecast(states, i, m, season, f, nmse, alpha_0, alpha_1, beta_0, beta_1)
        if math.fabs(f[0] - NA) < TOL:
            lik = NA
            return lik
        e[i - m] = y[i - m] - f[0]
        for j in range(nmse):
            if (i + j) < n:
                denom[j] += 1.
                tmp = y[i + j] - f[j]
                amse[j] = (amse[j] * (denom[j] - 1.0) + (tmp * tmp)) / denom[j]
        # update state
        update(states, i, m, season, alpha_0, alpha_1, beta_0, beta_1, y[i - m])
        lik = lik + e[i - m] * e[i - m]
        lik2 += math.log(math.fabs(f[0]))
    lik = n * math.log(lik)
    return lik

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def forecast(states, i, 
             m, season, 
             f, h, 
             alpha_0, alpha_1, beta_0, beta_1):
    # obs:
    # forecast are obtained in a recursive manner
    # this is not standard, for example in ets
    #forecasts
    new_states = np.zeros((m + h, states.shape[1]), dtype=np.float32)
    new_states[:m] = states[(i - m):i]
    for i_h in range(m, m + h):
        if season in [NONE, PARTIAL, FULL]:
            f[i_h - m] = new_states[i_h - 1, 0]
        else:
            f[i_h - m] = new_states[i_h - m, 0]
        if season > SIMPLE:
            f[i_h - m] += new_states[i_h - m, 2]
        update(new_states, i_h, m, season, alpha_0, alpha_1, beta_0, beta_1, f[i_h - m])

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def update(states, i,
           m, 
           season, # kind of season 
           alpha_0, alpha_1,
           beta_0, beta_1,
           y):
    # season
    if season in [NONE, PARTIAL, FULL]:
        e = y - states[i - 1, 0]
    else:
        e = y - states[i - m, 0]
    if season > SIMPLE:
        e -= states[i - m, 2]
        
    if season in [NONE, PARTIAL, FULL]:
        states[i, 0] = states[i - 1, 0] - (1. - alpha_1) * states[i - 1, 1] + (alpha_0 - alpha_1) * e
        states[i, 1] = states[i - 1, 0] + (1. - alpha_0) * states[i - 1, 1] + (alpha_0 + alpha_1) * e
    else:
        states[i, 0] = states[i - m, 0] - (1. - alpha_1) * states[i - m, 1] + (alpha_0 - alpha_1) * e
        states[i, 1] = states[i - m, 0] + (1. - alpha_0) * states[i - m, 1] + (alpha_0 + alpha_1) * e
    
    if season == PARTIAL:
        states[i, 2] = states[i - m, 2] + beta_0 * e
    if season == FULL:
        states[i, 2] = states[i - m, 2] - (1 - beta_1) * states[i - m, 3] + (beta_0 - beta_1) * e
        states[i, 3] = states[i - m, 2] + (1 - beta_0) * states[i - m, 3] + (beta_0 + beta_1) * e

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def cesforecast(x, states, m, season, 
                f, h, alpha_0, alpha_1, beta_0, beta_1):
    # compute forecasts
    n = len(x)
    m = 1 if season == NONE else m
    forecast(states=states, i=m + len(x), m=m, season=season, f=f, h=h, 
             alpha_0=alpha_0, alpha_1=alpha_1, 
             beta_0=beta_0, beta_1=beta_1) 

In [ ]:
#| hide
from statsforecast.utils import AirPassengers as ap

In [ ]:
#| hide
#nonseasonal test
nmse_ = len(ap)
amse_ = np.zeros(30)
e_ = np.zeros(len(ap))
alpha_0 = 2.001457
alpha_1 = 1.000727
beta_0 = 0.
beta_1 = 0.
init_states_non_seas = np.zeros((1 + len(ap), 2), dtype=np.float32)
init_states_non_seas[0, 0] = 112.0689
init_states_non_seas[0, 1] = 1301.9239
cescalc(y=ap, n=len(ap), 
        states=init_states_non_seas, m=12, 
        season=NONE, alpha_0=alpha_0, 
        alpha_1=alpha_1, beta_0=beta_0, 
        beta_1=beta_1,
        e=e_, amse=amse_, nmse=3)
np.testing.assert_array_equal(
    init_states_non_seas[[1, -1]],
    np.array([
        [  112.94645, -1191.9589 ],
        [  430.9216 ,  2040.1306 ]
    ], dtype=np.float32)
)

In [ ]:
#| hide
#nonseasonal forecast test
h = 13
fcsts = np.zeros(h, dtype=np.float32)
cesforecast(x=ap, states=init_states_non_seas, m=12, 
            season=NONE, 
            f=fcsts, h=h, 
            alpha_0=alpha_0, alpha_1=alpha_1, 
            beta_0=beta_0, beta_1=beta_1)
#taken from R using ces(AirPassengers, h=13)
np.testing.assert_array_almost_equal(
    fcsts,
    np.array([
        430.9211, 432.4049, 431.2324, 432.7212, 431.5439,
        433.0376, 431.8556, 433.3543, 432.1675, 433.6712,
        432.4796, 433.9884, 432.7920
    ], dtype=np.float32), 
    decimal=2
)

In [ ]:
#| hide
#simple seasonal test
nmse_ = len(ap)
amse_ = np.zeros(30)
lik_ = 0.
e_ = np.zeros(len(ap))
alpha_0 = 1.996411
alpha_1 = 1.206694
beta_0 = 0.
beta_1 = 0.
m = 12
init_states_s_ses = np.zeros((12 + len(ap), 2), dtype=np.float32)
init_states_s_ses[:m, 0] = np.array([
    130.4946, 137.3809, 148.8973, 146.0984, 135.8789, 156.9829, 168.8333, 170.1653,
    154.2067, 137.0842, 120.9946, 135.2192
])
init_states_s_ses[:m, 1] = np.array([
    36.59120,  81.54694,  29.20077,  43.01860, -26.30716, 108.47963, 129.26214,
    145.27629, 122.88351,  85.94281,  53.65530, 121.62019
])
cescalc(y=ap, n=len(ap), 
        states=init_states_s_ses, m=12, 
        season=SIMPLE, alpha_0=alpha_0, 
        alpha_1=alpha_1, beta_0=beta_0, 
        beta_1=beta_1,
        e=e_, amse=amse_, nmse=3)
np.testing.assert_array_equal(
    init_states_s_ses[[12, -1]],
    np.array([
        [123.45228 ,  34.794582],
        [505.3621  ,  95.29781 ]
    ], dtype=np.float32)
)

In [ ]:
#| hide
#simple seasonal forecast test
h = 13
fcsts = np.zeros(h, dtype=np.float32)
cesforecast(x=ap, states=init_states_s_ses, m=12, 
            season=SIMPLE, 
            f=fcsts, h=h, 
            alpha_0=alpha_0, alpha_1=alpha_1, 
            beta_0=beta_0, beta_1=beta_1)
#taken from R using ces(AirPassengers, h=13, seasonality = 'simple')
np.testing.assert_array_almost_equal(
    fcsts,
    np.array([
        446.2768, 423.5779, 481.4365, 514.7730, 533.5008,
        589.0500, 688.2703, 674.5891, 580.9486, 516.0776,
        449.7246, 505.3621, 507.9884
    ], dtype=np.float32), 
    decimal=2
)

In [ ]:
#| hide
#partial seasonal test
nmse_ = len(ap)
amse_ = np.zeros(30)
lik_ = 0.
e_ = np.zeros(len(ap))
alpha_0 = 1.476837
alpha_1 = 1.
beta_0 = 0.91997
beta_1 = 0.
m = 12
init_states_p_seas = np.zeros((12 + len(ap), 3), dtype=np.float32)
init_states_p_seas[:m, 0] = 122.5807
init_states_p_seas[:m, 1] = np.array([
    83.00356, 82.99918, 83.00837, 82.98910, 83.02952, 82.94475, 83.12252, 82.74971,
    83.53154, 81.89192, 85.33045, 78.11934
])
init_states_p_seas[:m, 2] = np.array([
    -9.7109728,  -5.5135757,   6.3941448,  -0.3545403,  -6.7264464,   8.2515199,
     19.6815401,  17.9080333,   2.7737274, -13.8587326, -22.0720405,  -4.6558357
])
cescalc(y=ap, n=len(ap), 
        states=init_states_p_seas, m=12, 
        season=2, alpha_0=alpha_0, 
        alpha_1=alpha_1, beta_0=beta_0, 
        beta_1=beta_1,
        e=e_, amse=amse_, nmse=3)
np.testing.assert_array_equal(
    init_states_p_seas[[12, -1]],
    np.array([
        [122.165985,  83.17633 , -10.511099],
        [438.5037  , 313.5582  ,  -7.581563]
    ], dtype=np.float32)
)

In [ ]:
#| hide
#partial seasonal forecast test
h = 13
fcsts = np.zeros(h, dtype=np.float32)
cesforecast(x=ap, states=init_states_p_seas, m=12, 
            season=PARTIAL, 
            f=fcsts, h=h, 
            alpha_0=alpha_0, alpha_1=alpha_1, 
            beta_0=beta_0, beta_1=beta_1)
#taken from R using ces(AirPassengers, h=13, seasonality = 'partial')
np.testing.assert_array_almost_equal(
    fcsts,
    np.array([
        437.6247, 412.9464, 445.5811, 498.5370, 493.0405, 550.7443, 
        629.2205, 607.1793, 512.3455, 462.1260, 383.4097, 430.9221, 437.6247
    ], dtype=np.float32), 
    decimal=2
)

In [ ]:
#| hide
#full seasonal test
nmse_ = len(ap)
amse_ = np.zeros(30)
lik_ = 0.
e_ = np.zeros(len(ap))
alpha_0 = 1.350795
alpha_1 = 1.009169
beta_0 = 1.777909
beta_1 = 0.973739
m = 12
init_states_f_seas = np.zeros((12 + len(ap), 4), dtype=np.float32)
init_states_f_seas[:m, 0] = np.array([
    227.7378, 226.2100, 224.6925, 223.1851, 221.6879, 220.2007, 218.7235, 217.2562,
    215.7987, 214.3511, 212.9129, 211.4852
])
init_states_f_seas[:m, 1] = np.array([
    167.7566, 166.6312, 165.5133, 164.4030, 163.3001, 162.2044, 161.1168, 160.0345,
    158.9649, 157.8874, 156.8592, 155.7203
])
init_states_f_seas[:m, 2] = np.array([
    -94.29597,  -89.38795,  -77.72370,  -82.64853,  -89.90721,  -75.64473,
    -63.48974,  -65.64998,  -78.81388,  -96.03032, -108.50292,  -91.61881
])
init_states_f_seas[:m, 3] = np.array([
    -39.621513,   17.702012,  -42.041080,  -59.691494, -139.156001,  -33.066817,
    -24.270666,   -4.161572,  -53.944371,  -62.401226,  -83.957612, -82.951031
])
cescalc(y=ap, n=len(ap), 
        states=init_states_f_seas, m=12, 
        season=3, alpha_0=alpha_0, 
        alpha_1=alpha_1, beta_0=beta_0, 
        beta_1=beta_1,
        e=e_, amse=amse_, nmse=3)
np.testing.assert_array_equal(
    init_states_f_seas[[12, -1]],
    np.array([
        [ 211.14023 ,  144.6129  ,  -97.42849 ,  -77.752975],
        [ 564.9002  ,  404.32245 , -130.90099 , -137.3275  ]
    ], dtype=np.float32)
)

In [ ]:
#| hide
#full seasonal forecast test
h = 13
fcsts = np.zeros(h, dtype=np.float32)
cesforecast(x=ap, states=init_states_f_seas, m=12, 
            season=FULL, 
            f=fcsts, h=h, 
            alpha_0=alpha_0, alpha_1=alpha_1, 
            beta_0=beta_0, beta_1=beta_1)
#taken from R using ces(AirPassengers, h=13, seasonality = 'full')
np.testing.assert_array_almost_equal(
    fcsts,
    np.array([
        450.9262, 429.2925, 465.4771, 510.1799, 517.9913, 578.5654,
        655.9219, 638.6218, 542.0985, 498.1064, 431.3293, 477.3273,
        501.3757
    ], dtype=np.float32), 
    decimal=2
)

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def initparam(alpha_0: float, alpha_1: float, 
              beta_0: float, beta_1: float,
              seasontype: str):
    if np.isnan(alpha_0):
        alpha_0 = 1.3
    if np.isnan(alpha_1):
        alpha_1 = 1.
    if seasontype == 'P':
        if np.isnan(beta_0):
            beta_0 = 0.1
    elif seasontype == 'F':
        if np.isnan(beta_0):
            beta_0 = 1.3
        if np.isnan(beta_1):
            beta_1 = 1.
    return {'alpha_0': alpha_0, 'alpha_1': alpha_1, 'beta_0': beta_0, 'beta_1': beta_1}

In [ ]:
#| hide
initparam(alpha_0=np.nan, alpha_1=np.nan, 
          beta_0=np.nan, beta_1=np.nan, 
          seasontype='N')

{'alpha_0': 1.3, 'alpha_1': 1.0, 'beta_0': nan, 'beta_1': nan}

In [ ]:
#| exporti
def initstate(y, m, seasontype):
    n = len(y)
    components = 2 + (seasontype == 'P') + 2 * (seasontype == 'F')
    lags = 1 if seasontype == 'N' else m
    states = np.zeros((lags, components), dtype=np.float32)
    if seasontype == 'N':
        idx = min(max(10, m), n)
        mean_ = np.mean(y[:idx])
        states[0, 0] = mean_
        states[0, 1] = mean_ / 1.1
    elif seasontype == 'S':
        states[:lags, 0] = y[:lags]
        states[:lags, 1] = y[:lags] / 1.1
    elif seasontype == 'P':
        states[:lags, 0] = np.mean(y[:lags])
        states[:lags, 1] = states[:lags, 0] / 1.1
        states[:lags, 2] = seasonal_decompose(y, period=lags).seasonal[:lags]
    elif seasontype == 'F':
        states[:lags, 0] = np.mean(y[:lags])
        states[:lags, 1] = states[:lags, 0] / 1.1
        states[:lags, 2] = seasonal_decompose(y, period=lags).seasonal[:lags]
        states[:lags, 3] = states[:lags, 2] / 1.1
    else:
        raise Exception(f'Unkwon seasontype: {seasontype}')
        
    return states

In [ ]:
#| hide
initstate(ap, 12, 'N')
initstate(ap, 12, 'S')
initstate(ap, 12, 'P')
initstate(ap, 12, 'F')

array([[126.666664 , 115.15151  , -24.748737 , -22.498852 ],
       [126.666664 , 115.15151  , -36.18813  , -32.8983   ],
       [126.666664 , 115.15151  ,  -2.2411616,  -2.0374196],
       [126.666664 , 115.15151  ,  -8.036616 ,  -7.3060145],
       [126.666664 , 115.15151  ,  -4.5063133,  -4.096648 ],
       [126.666664 , 115.15151  ,  35.40278  ,  32.184345 ],
       [126.666664 , 115.15151  ,  63.830807 ,  58.028004 ],
       [126.666664 , 115.15151  ,  62.82323  ,  57.112026 ],
       [126.666664 , 115.15151  ,  16.520203 ,  15.018366 ],
       [126.666664 , 115.15151  , -20.642677 , -18.76607  ],
       [126.666664 , 115.15151  , -53.593433 , -48.721302 ],
       [126.666664 , 115.15151  , -28.61995  , -26.018135 ]],
      dtype=float32)

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def switch(x: str):
    return {'N': 0, 'S': 1, 'P': 2, 'F': 3}[x]

In [ ]:
#| hide
switch('N')

0

In [ ]:
#| exporti
#@njit(nogil=NOGIL, cache=CACHE)
def pegelsresid_C(y: np.ndarray, 
                  m: int, 
                  init_state: np.ndarray, 
                  seasontype: str, 
                  alpha_0: float, alpha_1: float,
                  beta_0: float, beta_1: float, 
                  nmse: int):
    n = len(y)
    state_shape = init_states.shape
    states = np.full((state_shape[0] * (n + 1), state_shape[1]), fill_value=np.nan)
    states[:p] = init_state
    e = np.full_like(y, fill_value=np.nan)
    amse = np.full(nmse, fill_value=np.nan)
    lik = cescalc(y=y, n=n, states=states, m=m, 
                  season=switch(seasontype), 
                  alpha_0=alpha_0, alpha_1=alpha_1, 
                  beta_0=beta_0, beta_1=beta_1, e=e, 
                  amse=amse, nmse=nmse)
    if not np.isnan(lik):
        if np.abs(lik + 99999) < 1e-7:
            lik = np.nan
    return amse, e, states, lik

In [ ]:
#| export
#@njit(nogil=NOGIL, cache=CACHE)
def ces_target_fn(
        par,
        y,
        n,
        n_params,
        state_shape, 
        seasontype,
        nmse
    ):
    states = np.full((state_shape[0] * (n + 1), state_shape[1]), fill_value=np.nan)
    states[:state_shape[0]] = par[-(state_shape[0] * state_shape[1]):].reshape(state_shape)
    alpha_0, alpha_1 = par[:n_params] 
    e = np.full_like(y, fill_value=np.nan)
    amse = np.full(nmse, fill_value=np.nan)
    lik = cescalc(y=y, n=n, states=states, m=m, 
                  season=switch(seasontype), 
                  alpha_0=alpha_0, alpha_1=alpha_1, 
                  beta_0=beta_0, beta_1=beta_1, e=e, 
                  amse=amse, nmse=nmse)
    if lik < -1e10: 
        lik = -1e10 
    if math.isnan(lik): 
        lik = -np.inf
    if math.fabs(lik + 99999) < 1e-7: 
        lik = -np.inf
    return lik

In [ ]:
from scipy.optimize import minimize

In [ ]:
#| exporti
def optimize_ces_target_fn(
        x0, par, y, n, n_params, state_shape, 
        seasontype, nmse
    ):
    res = minimize(
        ces_target_fn, x0, 
        args=(y, n, n_params, state_shape, seasontype, nmse),
        method='Nelder-Mead'
    )
    return res

In [ ]:
#| exporti
def cesmodel(y: np.ndarray, m: int, 
             seasontype: str, 
             alpha_0: float, alpha_1: float,
             beta_0: float, beta_1: float, nmse: int):
    if seasontype == 'N':
        m = 1
    par_ = initparam(alpha_0, alpha_1, beta_1, beta_0, seasontype)
    init_state = initstate(y, m, seasontype)
    state_shape = init_state.shape
    par_ = {key: val for key, val in par_.items() if not np.isnan(val)}
    par = np.full(len(par_) + state_shape[0] * state_shape[1], fill_value=np.nan)
    par[:len(par_)] = list(par_.values())
    par[len(par_):] = init_state.flatten()
    fred = optimize_ces_target_fn(
        x0=par, par=par, y=y, n=len(y), n_params=len(par_), 
        state_shape=state_shape, seasontype=seasontype, nmse=nmse
    )
    fit_par = fred.x
    return fit_par

In [ ]:
#| hide
res = cesmodel(
    y=ap, m=12, seasontype='N',
    alpha_0=np.nan,
    alpha_1=np.nan,
    beta_0=np.nan, 
    beta_1=np.nan,
    nmse=3
)

In [ ]:
res

array([  2.28743533,   1.        ,  64.40458834, 155.41949262])

In [ ]:
#| exporti
def ets_f(y, m, model='ZZZ', 
          damped=None, alpha=None, beta=None, gamma=None, phi=None,
          additive_only=None, blambda=None, biasadj=None, 
          lower=np.array([0.0001, 0.0001, 0.0001, 0.8]), 
          upper=np.array([0.9999, 0.9999, 0.9999, 0.98]),
          opt_crit='lik', nmse=3, bounds='both',
          ic='aicc', restrict=True, allow_multiplicative_trend=False,
          use_initial_values=False, 
          maxit=2_000):
    # converting params to floats 
    # to improve numba compilation
    if alpha is None:
        alpha = np.nan
    if beta is None:
        beta = np.nan
    if gamma is None:
        gamma = np.nan
    if phi is None:
        phi = np.nan
    if blambda is not None:
        raise NotImplementedError('`blambda` not None')
    if nmse < 1 or nmse > 30:
        raise ValueError('nmse out of range')
    if any(upper < lower):
        raise ValueError('Lower limits must be less than upper limits')
    #refit model not implement yet
    errortype, trendtype, seasontype = model
    if errortype not in ['M', 'A', 'Z']:
        raise ValueError('Invalid error type')
    if trendtype not in ['N', 'A', 'M', 'Z']:
        raise ValueError('Invalid trend type')
    if seasontype not in ['N', 'A', 'M', 'Z']:
        raise ValueError('Invalid season type')
    if m < 1 or len(y) <= m:
        seasontype = 'M'
    if m == 1:
        if seasontype == 'A' or seasontype == 'M':
            raise ValueError('Nonseasonal data')
        else:
            #model[3] = 'N'
            seasontype = 'N'
    if m > 24:
        if seasontype in ['A', 'M']:
            raise ValueError('Frequency too high')
        elif seasontype == 'Z':
            warnings.warn(
                "I can't handle data with frequency greater than 24. " 
                "Seasonality will be ignored."
            )
            #model[3] = 'N'
            seasontype = 'N'
    if restrict:
        if (errortype == 'A' and (trendtype == 'M' or seasontype == 'M')) \
            or (errortype == 'M' and trendtype == 'M' and seasontype == 'A') \
            or (additive_only and (errortype == 'M' or trendtype == 'M' or seasontype == 'M')):
            raise ValueError('Forbidden model combination')
    data_positive = min(y) > 0
    if (not data_positive) and errortype == 'M':
        raise ValueError('Inappropriate model for data with negative or zero values')
    if damped is not None:
        if damped and trendtype=='N':
            ValueError('Forbidden model combination')
    n = len(y)
    npars = 2 # alpha + l0
    if trendtype in ['A', 'M']:
        npars += 2 #beta + b0
    if seasontype in ['A', 'M']:
        npars += 2 # gamma + s
    if damped is not None:
        npars += damped
    #ses for non-optimized tiny datasets
    if n <= npars + 4:
        #we need HoltWintersZZ function
        raise NotImplementedError('tiny datasets')
    # fit model (assuming only one nonseasonal model)
    if errortype == 'Z':
        errortype = ['A', 'M']
    if trendtype == 'Z':
        trendtype = ['N', 'A']
        if allow_multiplicative_trend:
             trendtype += ['M']
    if seasontype == 'Z':
        seasontype = ['N', 'A', 'M']
    if damped is None:
        damped = [True, False]
    else:
        damped = [damped]
    best_ic = np.inf
    for etype in errortype:
        for ttype in trendtype:
            for stype in seasontype:
                for dtype in damped:
                    if ttype == 'N' and dtype:
                        continue
                    if restrict:
                        if etype == 'A' and (ttype == 'M' and stype == 'M'):
                            continue
                        if etype == 'M' and ttype == 'M' and stype == 'A':
                            continue
                        if additive_only and (etype == 'M' or ttype == 'M' or stype == 'M'):
                            continue
                    if (not data_positive) and etype == 'M':
                        continue
                    if stype != 'N' and m == 1:
                        continue
                    fit = etsmodel(y, m, etype, ttype, stype, dtype,
                                   alpha, beta, gamma, phi,
                                   lower=lower, upper=upper, opt_crit=opt_crit,
                                   nmse=nmse, bounds=bounds, 
                                   maxit=maxit)
                    fit_ic = fit[ic]
                    if not np.isnan(fit_ic):
                        if fit_ic < best_ic:
                            model = fit
                            best_ic = fit_ic
                            best_e = etype
                            best_t = ttype
                            best_s = stype
                            best_d = dtype
    if np.isinf(best_ic):
        raise Exception('no model able to be fitted')
    model['method'] = f"ETS({best_e},{best_t}{'d' if best_d else ''},{best_s})"
    return model

In [ ]:
#| exporti
def pegelsfcast_C(h, obj, npaths=None, level=None, bootstrap=None):
    forecast = np.full(h, fill_value=np.nan)
    states = obj['states'][-1,:]
    etype, ttype, stype = [switch(comp) for comp in obj['components'][:3]]
    phi = 1 if obj['components'][3] == 'N' else obj['par'][3]
    m = obj['m']
    etsforecast(x=states, m=m, trend=ttype, season=stype, 
                phi=phi, h=h, f=forecast)
    return forecast

In [ ]:
#| exporti
def forecast_ets(obj, h):
    fcst = pegelsfcast_C(h, obj)
    out = {'mean': fcst}
    out['residuals'] = obj['residuals']
    out['fitted'] = obj['fitted']
    return out

In [ ]:
#| hide
import matplotlib.pyplot as plt
res = ets_f(ap, m=1)
fcst = forecast_ets(res, 12)
plt.plot(np.arange(0, len(ap)), ap)
plt.plot(np.arange(len(ap), len(ap) + 12), fcst['mean'])